In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator

import sys; sys.path.append("../../")
from DataPreprocessing.DataPreprocessing import read_data, remove_useless_col

# Spark session

In [2]:
# spark = SparkSession.builder.appName('InstallationsPrediction').getOrCreate()

spark = SparkSession.builder \
    .master('local[*]') \
    .config("spark.driver.memory", "32g") \
    .appName('InstallationsPrediction') \
    .getOrCreate()

sc=spark.sparkContext

# Read the data

In [3]:
interesting_categ_cols= ['Category', 'Ad Supported', 'In App Purchases', 'Editors Choice',\
                  'Free', 'Size', "Minimum Android","Developer Id","Developer Email"]

df= read_data(spark,file_name='train',encode=True,cols_to_encode=interesting_categ_cols)

In [4]:
df.dtypes

[('App Name', 'string'),
 ('App Id', 'string'),
 ('Rating', 'float'),
 ('Rating Count', 'int'),
 ('Installs', 'string'),
 ('Minimum Installs', 'int'),
 ('Maximum Installs', 'int'),
 ('Price', 'float'),
 ('Released', 'string'),
 ('Last Updated', 'string'),
 ('Content Rating', 'string'),
 ('Category', 'double'),
 ('Ad Supported', 'double'),
 ('In App Purchases', 'double'),
 ('Editors Choice', 'double'),
 ('Free', 'double'),
 ('Size', 'double'),
 ('Minimum Android', 'double'),
 ('Developer Id', 'double'),
 ('Developer Email', 'double')]

## Features 

In [5]:
'''
App Name, App Id, Rating, Rating Count, Released, Last Updated, Content Rating are not interesting here for max installations prediction,
so we can remove them safely
'''

useless_cols= ['App Name','App Id','Rating','Rating Count','Released','Last Updated','Content Rating','Installs','Minimum Installs']

df= remove_useless_col(df,useless_cols)

In [6]:
df.dtypes

[('Maximum Installs', 'int'),
 ('Price', 'float'),
 ('Category', 'double'),
 ('Ad Supported', 'double'),
 ('In App Purchases', 'double'),
 ('Editors Choice', 'double'),
 ('Free', 'double'),
 ('Size', 'double'),
 ('Minimum Android', 'double'),
 ('Developer Id', 'double'),
 ('Developer Email', 'double')]

In [7]:
#collect all features in one column except Maximum Installs
assembler = VectorAssembler(inputCols=df.columns[1:],outputCol="Features")
vec = assembler.transform(df)

In [11]:
train_df= vec.select("Features","Maximum Installs")
train_df.show(5)

+--------------------+----------------+
|            Features|Maximum Installs|
+--------------------+----------------+
|(10,[1,6,8,9],[4....|            3987|
|(10,[1,6,7,8,9],[...|               7|
|(10,[1,2,6,8,9],[...|             279|
|(10,[6,7,8,9],[35...|               9|
|(10,[1,6,7,8,9],[...|             305|
+--------------------+----------------+
only showing top 5 rows



# Initiate the model

In [12]:
lr = LinearRegression(featuresCol='Features',labelCol='Maximum Installs')
lr_model = lr.fit(train_df)

Py4JJavaError: An error occurred while calling o577.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 7 in stage 32.0 failed 1 times, most recent failure: Lost task 7.0 in stage 32.0 (TID 100) (Mariem executor driver): java.lang.OutOfMemoryError: Java heap space

Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2785)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2721)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2720)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2720)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTaskSetFailed$1(DAGSchedule

## Training Score

In [ ]:
trainingSummary = lr_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)